# Regressor Chain

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# fetch yahoo data
import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2018-08-27'

# Read data 
dataset = yf.download(symbol,start,end)

# Only keep close columns 
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2014-01-02,3.95,3.95,3.98,3.84,3.85,20548400
2014-01-03,4.00,4.00,4.00,3.88,3.98,22887200
2014-01-06,4.13,4.13,4.18,3.99,4.01,42398300
2014-01-07,4.18,4.18,4.25,4.11,4.19,42932100
2014-01-08,4.18,4.18,4.26,4.14,4.23,30678700


In [3]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()

In [4]:
dataset.tail()

,Adj Close,Close,High,Low,Open,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2018-08-20,19.980000,19.980000,20.08,19.350000,19.790001,62983200,0,1,1,0.010622
2018-08-21,20.400000,20.400000,20.42,19.860001,19.980000,55629000,1,1,1,0.021021
2018-08-22,20.900000,20.900000,20.92,20.209999,20.280001,62002700,1,1,1,0.024510
2018-08-23,22.290001,22.290001,22.32,21.139999,21.190001,113444100,1,1,1,0.066507
2018-08-24,23.980000,23.980000,24.00,22.670000,22.910000,164328200,0,0,0,0.075819


In [5]:
X = dataset.drop(['Adj Close', 'Open'], axis=1)
Y = dataset[['Adj Close', 'Open']]

In [6]:
print(X.shape, Y.shape)

(1170, 8) (1170, 2)


In [7]:
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR

model = LinearSVR()
wrapper = RegressorChain(model)
wrapper.fit(X, Y)

RegressorChain(base_estimator=LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0),
        cv=None, order=None, random_state=None)

In [8]:
print(dataset.describe())

         Adj Close        Close         High          Low         Open  \
count  1170.000000  1170.000000  1170.000000  1170.000000  1170.000000   
mean      7.002803     7.002803     7.146650     6.855419     7.001342   
std       4.861608     4.861608     4.953517     4.754049     4.855620   
min       1.620000     1.620000     1.690000     1.610000     1.620000   
25%       2.702500     2.702500     2.780000     2.660000     2.710000   
50%       4.275000     4.275000     4.350000     4.175000     4.250000   
75%      11.550000    11.550000    11.775000    11.307500    11.572500   
max      23.980000    23.980000    24.000000    22.670000    22.910000   

             Volume  Increase_Decrease  Buy_Sell_on_Open     Buy_Sell  \
count  1.170000e+03        1170.000000       1170.000000  1170.000000   
mean   3.763371e+07           0.454701          0.505128     0.499145   
std    3.355409e+07           0.498157          0.500188     0.500213   
min    0.000000e+00           0.000000   

In [9]:
# input the data_in based on statistics
data_in = [[23.98, 22.91, 7.00, 7.00, 1.62, 1.62, 4.27, 4.25]]
yhat = wrapper.predict(data_in)

In [10]:
# summarize prediction
print(yhat[0])

[16.72625136 16.72625136]


In [11]:
wrapper.score(X, Y)

0.9838719181304376